# Ch35t


## Parser

In [1]:
# parser is used to parse a chest's json description
import parser

# if a Parser is instantiated with no input params, it
# will have an empty json
cp = parser.Parser()
print(cp.json())

None


## Manually load a chest
### From file

In [2]:
cp._load_file("chest_01.json")
cp.json()

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'format': 'text/plain',
  'method': 'No method'}}

### From URL

In [3]:
url = "http://3564020356.org/zelif/chest_01.json"
cp._load_url(url)
cp.json()

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'format': 'text/plain',
  'method': 'No method'}}

### From string

In [4]:
### NOTE that method was a string before, here we use a dictionary
### TODO: do we want to use both or always dictionary?
jj = '''
{
"hint": { "data": "this is the hint", "format": "text/plain" },
"payload": {"data": "this is the payload", 
  "format": "text/plain",
  "method": {"name": "this is the method"}
  }
}
'''
cp._load_string(jj)
cp.json()

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'format': 'text/plain',
  'method': {'name': 'this is the method'}}}

*NOTE* that manual load won't probably be required. If OCParser is initalised with a string that starts with `http://` or `https://` it will automatically call load_url, if it starts with `file://` it will automatically load a file, otherwise it will load a string. As we might want a URL to be considered as a string and not as a reference to some content, it is still possible to do that by instantiating an emtpy parser and loading the url as a string.

## Load automatically
Parser is able to automatically detect whether the JSON chest description is being passed as an URL, a file, or a string

In [5]:
cp = parser.Parser('{"hint": {"data": "This is a hint"}, "payload": {"data": "this is the payload"}}')
cp.json()

{'hint': {'data': 'This is a hint'},
 'payload': {'data': 'this is the payload'}}

In [6]:
cp = parser.Parser("http://3564020356.org/zelif/chest_01.json")
cp.json()

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'format': 'text/plain',
  'method': 'No method'}}

In [7]:
cp = parser.Parser("file://chest_01.json")
cp.json()

{'hint': {'data': 'this is the hint', 'format': 'text/plain'},
 'payload': {'data': 'this is the payload',
  'format': 'text/plain',
  'method': 'No method'}}

In [8]:
print(cp.hint())
print(cp.payload())

[i] Hint
    Origin: None
    Data: this is the hint
    Format: text/plain

[i] Payload
    Origin: None
    Data: this is the payload
    Format: text/plain
    Method: No method



# Build hint object

Every hint is characterised by some `data` content and a `format`. Depending on the format, the hint will be interpreted in different ways (see e.g. https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/MIME_types/Common_types):

* `text/plain`: data can be simply printed
* `text/html`: html that needs to be interpreted/shown properly
* `image/*`: data will be displayed by the appropriate application
* `application/zip`: a zip file, to be unpacked in the chest directory
* `application/octet-stream`: this is a generic file (perhaps one wants to make the hint a riddle too?)

NOTE that:

* Data URLs (https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URLs) can be a good hint on how to embed data which is not just plain text, define mimetype, etc
* if we want to have BOTH online and offline stuff, we could have an `origin` for URL origins, `format` for their remote formats, `data` for the actual data that is downloaded from the URL and base64-encoded to allow for offline access.

SO:
* the fields under `hint` are `origin`, `format`, and `data`
* if we want to specify a hint by reference we provide `origin` (any URL) and `format` (the format of the data that will be returned by that URL, if the app has to parse it)
* if we want to specify a hint by value we provide `data` in a way that recalls data URLs, i.e. as in
```
data:[<mediatype>][;base64],<data>
```
* `mediatype` is specified as `format` and if `origin` and `data` are both present they share the same mediatype. Also format is, by default, `text/plain` (perhaps do `text/plain;charset=US-ASCII`?)
* `base64` is provided at the beginning of `data` optional if we want the data to be provided as encoded (great if we have binaries)
* `data` is the actual data. Differently from data URLs, it is NOT provided as URI-encoded

In [11]:
import hint as h

jj = '''{
"hint": { 
  "data": "this is the hint", 
  "format": "text/plain" }
}'''

cp._load_string(jj)
hint = h.Hint(cp.json())
print(hint.data)
print(hint.format)

### note that format is text/plain by default
jj = '''{
"hint": { 
  "data": "this is the hint"
}
}'''

cp._load_string(jj)
hint = h.Hint(cp.json())
print(hint.data)
print(hint.format)

this is the hint
text/plain
this is the hint
text/plain


In [12]:
jj = '''{
"hint": { 
  "origin": "http://3564020356.org/deserve.htm",
  "format": "text/html"
}
}'''

cp._load_string(jj)
print(cp.json())
hint = h.Hint(cp.json())
print(hint)

{'hint': {'origin': 'http://3564020356.org/deserve.htm', 'format': 'text/html'}}
[i] Hint
    Origin: http://3564020356.org/deserve.htm
    Data: None
    Format: text/html



# Build payload

In [13]:
import payload as p

jj = '''{
"payload": { 
  "data": "this is the payload"
}
}'''

cp._load_string(jj)
cp.json()
payload = p.Payload(cp.json())
print(payload.data)
print(payload.format)

this is the payload
text/plain


* TODO: add code for input- and output- formats

In [14]:
print(payload)

[i] Payload
    Origin: None
    Data: this is the payload
    Format: text/plain
    Method: None



In [15]:
cp = parser.Parser("file://deserve.json")
cp.json()
print(cp.hint())
print(cp.payload())

[i] Hint
    Origin: None
    Data: MAL TIRRUEZF CR MAL RKZYIOL EX MAL OIY UAE RICF "MAL ACWALRM DYEUPLFWL CR ME DYEU MAIM UL IZL RKZZEKYFLF GH OHRMLZH"
    Format: text/plain

[i] Payload
    Origin: None
    Data: b535ffd9263f275c6747ec804fee198e
    Format: text/plain
    Method: md5



In [16]:
%load_ext autoreload
%autoreload 2

In [21]:
from ch35t import Chest

c = Chest("file://riddle02.json", chests_dir="./chests")
c.show_hint()

Files are available in ./chests:
riddle02/
riddle02/+Ma's Reversing Riddle 02_files/
riddle02/+Ma's Reversing Riddle 02_files/0c.zip
riddle02/+Ma's Reversing Riddle 02_files/alice.gif
riddle02/+Ma's Reversing Riddle 02_files/skull02.jpg
riddle02/+Ma's Reversing Riddle 02.html



In [23]:
c.unlock()

 whatever


Wrong key
